In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('C:\code\qdc2')
import matplotlib.pyplot as plt 
import os 
import numpy as np 
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.qdc_experiment import QDCExperiment
from qdc.misc import tnow
DATA_DIR = os.path.join(os.path.abspath(os.path.curdir), 'data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
is_step_index = False
fiber_L = 0.1e6  if is_step_index else 0.5e6 # um 
N_wl = 81
N_classical = 20
N_SPDC = 20
wl0 = 0.810
Dwl = 0.020 if is_step_index else 0.080
# dzs = [80, 20, 0]
dzs = [0]

s = ManyWavelengthFiber(wl0=wl0, Dwl=Dwl, N_wl=N_wl, fiber_L=fiber_L, rng_seed=5555, is_step_index=is_step_index)
s.gaussian_params = np.array([8, 0, 0, 0.0, 0.0])
s.gaussian_dparams = np.array([0, 12, 12, 0.5, 0.5])

exp = QDCExperiment(s)
res = exp.run_PCCs_different_dz(dzs=dzs, N_classical=N_classical, N_SPDC=N_SPDC)
res.show(title='')
res.show_incoherent_sum()

f_type = 'SI' if is_step_index else 'GRIN'
saveto_path = rf'{DATA_DIR}\\{tnow()}_{f_type}_{round(fiber_L*1e-6, 2)}m.npz'
res.saveto(saveto_path)

Getting 81 fibers...


81it [00:05, 14.15it/s]


Got fibers!
Getting classical with average on 20 ...


Running classical measurements: 100%|██████████| 20/20 [01:56<00:00,  5.85s/it]


Getting SPDC with average on 20, dz=0 ...


Running SPDC measurements:  20%|██        | 4/20 [00:47<03:08, 11.79s/it]

In [88]:
# res.show(title='')
iter_no = 3
dz = 0
# res.show_incoherent_sum(iter_no=iter_no, dz=dz)
res.metadata["g_params_list"]# [iter_no]

[array([ 8.        , -5.        ,  8.        ,  0.44197532,  0.46811559]),
 array([ 8.00000000e+00,  8.00000000e+00,  1.20000000e+01, -2.91142569e-01,
        -5.44380474e-03]),
 array([ 8.        , -7.        , -2.        ,  0.23808436,  0.32522094]),
 array([ 8.00000000e+00,  1.10000000e+01,  0.00000000e+00, -9.53795383e-02,
        -6.03170194e-03]),
 array([ 8.        , -2.        ,  7.        , -0.02780333,  0.43897139])]

In [52]:
f = s.fibers[0]
f.set_input_gaussian(sigma=8, X0=3, Y0=8, X_linphase=0.3, Y_linphase=0.3, random_phase=0.0)
f.propagate(False)

fig, axes = plt.subplot_mosaic(
    [["left", "right"],
     ["bottom", "bottom"]],
    figsize=(9, 6)
)

from matplotlib import patches
# make circle centered in the middle of the plot
circle = patches.Circle((f.npoints//2, f.npoints//2), 25/f.index_profile.dh, color='red', fill=False, linestyle='dashed')
axes["left"].add_patch(circle)
f.show_profile(f.profile_0,   ax=axes["left"],   title='input')
f.show_profile(f.profile_end, ax=axes["right"],  title='output')
axes["bottom"].plot(np.abs(f.modes_0)**2)
plt.tight_layout()
fig.show()


In [36]:
f.show_mode(20)

In [6]:
s = ManyWavelengthFiber(wl0=wl0, Dwl=Dwl, N_wl=N_wl, fiber_L=fiber_L, rng_seed=5555, is_step_index=is_step_index)

Getting 81 fibers...


81it [00:10,  8.05it/s]


Got fibers!
